In [9]:
import pandas as pd
import openpyxl

In [10]:
# import election results for state senate district 9
district9_results = pd.read_excel('detail.xlsx', sheet_name='2', skiprows=1)

/Users/amahaja/GitHub/rehmet_election/.venv/lib/python3.12/site-packages/openpyxl/styles/stylesheet.py:237: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")


In [11]:
district9_results.head()

,Unnamed: 0,Unnamed: 1,Taylor Rehmet,Unnamed: 3,Unnamed: 4,Unnamed: 5,Leigh Wambsganss,Unnamed: 7,Unnamed: 8,Unnamed: 9,Unnamed: 10
0,Precinct,Registered Voters,Absentee,Early,Election,Total Votes,Absentee,Early,Election,Total Votes,Total
1,1001,1940,3,99,98,200,1,43,45,89,289
2,1005,942,3,21,51,75,0,1,2,3,78
3,1008,314,1,12,24,37,0,0,6,6,43
4,1010,698,1,0,20,21,0,2,0,2,23


In [12]:
# there's a total row we don't want at the end, so let's drop that
district9_results.tail()

,Unnamed: 0,Unnamed: 1,Taylor Rehmet,Unnamed: 3,Unnamed: 4,Unnamed: 5,Leigh Wambsganss,Unnamed: 7,Unnamed: 8,Unnamed: 9,Unnamed: 10
377,4836,3126,10,76,123,209,4,67,56,127,336
378,4844,1820,2,40,73,115,0,29,44,73,188
379,4846,1720,3,49,39,91,2,16,33,51,142
380,4849,3336,1,83,137,221,0,39,81,120,341
381,Total:,617753,1115,24571,28594,54280,608,19375,20617,40600,94880


In [13]:
# drop last row
district9_results = district9_results.drop(380).reset_index(drop=True)

In [14]:
# how many rows in the dataframe?
len(district9_results)

381

In [15]:
# rename columns to combine candidate names with vote types -- essentially flatten since there are multiple levels of headers
district9_results.columns = [
    'Precinct',
    'Registered_Voters',
    'absentee_TaylorRehmet',
    'early_TaylorRehmet',
    'election_TaylorRehmet',
    'totalVotes_TaylorRehmet',
    'absentee_LeighWambsganss',
    'early_LeighWambsganss',
    'election_LeighWambsganss',
    'totalVotes_LeighWambsganss',
    'Total'
]

# drop first row so we only have one row of headers
district9_results = district9_results.drop(0).reset_index(drop=True)

district9_results.head()

,Precinct,Registered_Voters,absentee_TaylorRehmet,early_TaylorRehmet,election_TaylorRehmet,totalVotes_TaylorRehmet,absentee_LeighWambsganss,early_LeighWambsganss,election_LeighWambsganss,totalVotes_LeighWambsganss,Total
0,1001,1940,3,99,98,200,1,43,45,89,289
1,1005,942,3,21,51,75,0,1,2,3,78
2,1008,314,1,12,24,37,0,0,6,6,43
3,1010,698,1,0,20,21,0,2,0,2,23
4,1011,1134,3,19,45,67,0,2,3,5,72


In [16]:
# verify types
district9_results.dtypes

Precinct                         str
Registered_Voters             object
absentee_TaylorRehmet         object
early_TaylorRehmet            object
election_TaylorRehmet         object
totalVotes_TaylorRehmet       object
absentee_LeighWambsganss      object
early_LeighWambsganss         object
election_LeighWambsganss      object
totalVotes_LeighWambsganss    object
Total                         object
dtype: object

In [ ]:
# change type to string to merge
district9_results['Precinct'] = district9_results['Precinct'].astype(str)

In [18]:
# read in vtd census demographics data
vtd_census_2020 = pd.read_csv('vtd_census - VTDs_24PG_Pop.csv')

In [19]:
len(vtd_census_2020)

9712

In [20]:
# look at types 
vtd_census_2020.dtypes

CountyFIPS    int64
County          str
CNTY          int64
VTD             str
CNTYVTD         str
VTDKEY        int64
total         int64
anglo         int64
nanglo        int64
asian         int64
black         int64
hisp          int64
bh            int64
vap           int64
anglovap      int64
nanglovap     int64
asianvap      int64
blackvap      int64
hispvap       int64
bhvap         int64
dtype: object

In [21]:
# ensure type is string
vtd_census_2020['VTD'] = vtd_census_2020['VTD'].astype(str)


In [22]:
# join both data frames on Precinct and VTD
# the key to join is NOT vtdkey -- values for that are as low as 1. there's no precinct 1 in district 9

merged_df = pd.merge(district9_results, vtd_census_2020, left_on='Precinct', right_on='VTD', how='inner')

In [23]:
# there are multiple rows for each precinct after you join on vtd
# after further inspection, it's because there are multiple vtds per precinct because the data encompasses all of texas
# for example, there's a precinct/vtd "1001" in BASTROP, BEXAR, CAMP, DENTON, ELLIS, HARRIS, and TARRANT counties. 
# but many of these are nowhere near state senate district 9.
merged_df

,Precinct,Registered_Voters,absentee_TaylorRehmet,early_TaylorRehmet,election_TaylorRehmet,totalVotes_TaylorRehmet,absentee_LeighWambsganss,early_LeighWambsganss,election_LeighWambsganss,totalVotes_LeighWambsganss,...,black,hisp,bh,vap,anglovap,nanglovap,asianvap,blackvap,hispvap,bhvap
0,1001,1940,3,99,98,200,1,43,45,89,...,17,61,77,387,304,83,10,13,46,59
1,1001,1940,3,99,98,200,1,43,45,89,...,21,352,366,552,207,345,10,15,312,325
2,1001,1940,3,99,98,200,1,43,45,89,...,491,998,1464,2319,1291,1028,17,364,598,953
3,1001,1940,3,99,98,200,1,43,45,89,...,77,478,554,1837,1410,427,14,47,296,342
4,1001,1940,3,99,98,200,1,43,45,89,...,1772,1019,2727,2443,550,1893,23,1209,613,1812
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
652,4830,1728,2,46,57,105,6,60,98,164,...,90,314,400,1417,1076,341,36,61,188,248
653,4832,1774,0,61,76,137,0,27,26,53,...,149,344,487,784,399,385,37,99,209,305
654,4833,2116,2,78,114,194,1,100,108,209,...,158,293,444,1524,1115,409,56,105,197,301
655,4836,3126,10,76,123,209,4,67,56,127,...,1187,1550,2668,4070,1694,2376,566,723,1022,1719


In [24]:
# let's ensure we only have information for counties within state senate district 9
# district 9 is only in tarrant county

merged_df = merged_df[merged_df['County'] == 'TARRANT']

In [25]:
# are there any duplicate precincts now?
merged_df.duplicated(subset=['Precinct']).sum()

np.int64(0)

In [26]:
# seems like 6 precincts were dropped?
len(merged_df)

373

In [27]:
# show precincts in district9_results but not in merged_df
precincts_in_original = set(district9_results['Precinct'])
precincts_in_merged = set(merged_df['Precinct'])
missing_precincts = precincts_in_original - precincts_in_merged

print(f"Total precincts in district9_results: {len(precincts_in_original)}")
print(f"Total precincts in merged_df: {len(precincts_in_merged)}")
print(f"Precincts not in merged: {len(missing_precincts)}")
if missing_precincts:
    print("Missing precincts:")
    for p in sorted(missing_precincts):
        print(f"  {p}")

Total precincts in district9_results: 380
Total precincts in merged_df: 373
Precincts not in merged: 7
Missing precincts:
  1823
  3325
  4245
  4598
  4737
  4846
  Total:


In [28]:
# check for nas
merged_df.isna().sum()

Precinct                      0
Registered_Voters             0
absentee_TaylorRehmet         0
early_TaylorRehmet            0
election_TaylorRehmet         0
totalVotes_TaylorRehmet       0
absentee_LeighWambsganss      0
early_LeighWambsganss         0
election_LeighWambsganss      0
totalVotes_LeighWambsganss    0
Total                         0
CountyFIPS                    0
County                        0
CNTY                          0
VTD                           0
CNTYVTD                       0
VTDKEY                        0
total                         0
anglo                         0
nanglo                        0
asian                         0
black                         0
hisp                          0
bh                            0
vap                           0
anglovap                      0
nanglovap                     0
asianvap                      0
blackvap                      0
hispvap                       0
bhvap                         0
dtype: i

In [29]:
# export csv
merged_df.to_csv('district9_election_census_2020.csv', index=False)